# ✅Gerekli Kütüphanelerin İçe Aktarılması

In [1]:
import numpy as np  # NumPy, özellikle vektör ve matris işlemleri için kullanılır.
import gym  # OpenAI Gym, RL ortamlarını sağlar. Burada CartPole kullanılıyor.
from collections import deque  # Deneyim hafızası için çift taraflı kuyruk.
from tensorflow.keras.models import Sequential  # Basit katmanlı model tanımı.
from tensorflow.keras.layers import Dense  # Yapay sinir ağına tam bağlantılı katmanlar ekler.
from tensorflow.keras.optimizers import Adam  # Geri yayılımda kullanılan optimizasyon algoritması.
import random  # Rastgele eylem seçimi ve örnekleme için kullanılır.
from tqdm import tqdm  # Eğitim döngüsü ilerlemesini görsel olarak takip etmeye yarar.

# ✅ Derin Q-Learning Ajanının Tanımı

In [2]:
class DQLAgent:
    def __init__(self, env):
        self.state_size = env.observation_space.shape[0]  # Ortamın gözlem (state) boyutu alınır.
        self.action_size = env.action_space.n  # Ortamda yapılabilecek toplam eylem sayısı.

        self.gamma = 0.95  # Gelecekteki ödüllerin bugüne indirgenme katsayısı (discount factor).
        self.learning_rate = 0.001  # Öğrenme oranı.

        self.epsilon = 1  # Başlangıçta tamamen rastgele eylem seçimi (exploration).
        self.epsilon_decay = 0.995  # Her bölüm sonunda epsilon değeri yavaşça azaltılır.
        self.epsilon_min = 0.01  # Epsilon’un ulaşabileceği en düşük değer.

        self.memory = deque(maxlen=1000)  # Deneyimlerin saklanacağı bir kuyruk (Experience Replay).

        self.model = self.build_model()  # Sinir ağı modeli oluşturulur.

    # Sinir Ağı Modelinin İnşası
    def build_model(self):
        model = Sequential()
        
        model.add(Dense(48, input_dim=self.state_size, activation="relu"))  # İlk katman: Girdi boyutuna göre nöronlar.
        
        model.add(Dense(24, activation="relu"))  # Orta katman: daha az nöron ile temsil gücü.
       
        model.add(Dense(self.action_size, activation="linear"))  # Çıkış katmanı: Her aksiyon için bir değer.
        
        model.compile(loss="mse", optimizer=Adam(learning_rate=self.learning_rate))  # Hata fonksiyonu ve optimizer.
        return model

    # Hafızaya Deneyim Ekleme Fonksiyonu
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))  # Her adımı belleğe kaydeder.

    # Aksiyon Seçme Fonksiyonu (ε-Greedy Politikası)
    def act(self, state):
        if random.uniform(0, 1) <= self.epsilon:
            return env.action_space.sample()  # Rastgele aksiyon seç (exploration).
        
        act_values = self.model.predict(state, verbose=0)  # Modelle tahmin yap (exploitation).
        return np.argmax(act_values[0])  # En yüksek Q-değerine sahip aksiyon seçilir.

    # Replay - Ajanın Öğrenme Fonksiyonu
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return  # Yeterli deneyim yoksa öğrenme yapılmaz.

        minibatch = random.sample(self.memory, batch_size)  # Hafızadan rastgele örnekler seçilir.

        for state, action, reward, next_state, done in minibatch:
            if done:
                target = reward  # Eğer bölüm bittiyse sadece ödül kullanılır.
            else:
                target = reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0])
                # Eğer devam ediyorsa gelecekteki maksimum ödül eklenir.

            train_target = self.model.predict(state, verbose=0)
            train_target[0][action] = target  # Sadece ilgili aksiyonun Q-değeri güncellenir.

            self.model.fit(state, train_target, verbose=0)  # Model eğitilir.

        
    # Epsilon Güncelleme Fonksiyonu (Exploration Azaltımı)
    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay  # Epsilon zamanla azaltılır.

# ✅ Ana Eğitim Döngüsü - Ajanın Ortamda Öğrenmesi

In [3]:
env = gym.make("CartPole-v1", render_mode="human")  # Ortam oluşturulur (insan gözlemi için).
agent = DQLAgent(env)

batch_size = 32  # Mini-batch boyutu.
episodes = 3  # Eğitim bölümü sayısı (daha sonra artırılabilir).

for e in tqdm(range(episodes)):
    state = env.reset()[0]  # Ortam sıfırlanır ve başlangıç durumu alınır.
    state = np.reshape(state, [1, 4])  # Modelle uyumlu hale getirilir.

    time = 0  # Adım sayacı

    while True:
        action = agent.act(state)  # Ajan aksiyon seçer.

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated  # Bölümün bitip bitmediği kontrol edilir.
        next_state = np.reshape(next_state, [1, 4])  # Modelle uyumlu hale getirilir.

        agent.remember(state, action, reward, next_state, done)  # Hafızaya kaydedilir.
        state = next_state  # Yeni duruma geçilir.

        agent.replay(batch_size)  # Öğrenme yapılır.
        agent.adaptiveEGreedy()  # Epsilon güncellenir.

        time += 1

        if done:
            print(f"Episode {e+1} finished after {time} timesteps")  # Bölüm sonucu yazdırılır.
            break


C:\Users\Teymur Mammadov\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Teymur Mammadov\anaconda3\Lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
 33%|███▎      | 1/3 [00:02<00:04,  2.10s/it]

Episode 1 finished after 23 timesteps


 67%|██████▋   | 2/3 [00:42<00:24, 24.65s/it]

Episode 2 finished after 11 timesteps


100%|██████████| 3/3 [03:57<00:00, 79.11s/it] 

Episode 3 finished after 16 timesteps


In [ ]:
# Let the agent learn to balance! 🤖